## Notebook to upload Examples in the Excel

In [8]:
import pandas as pd
import os
from openai import OpenAI
import sklearn as sk
from sklearn.metrics.pairwise import euclidean_distances
from dotenv import load_dotenv
load_dotenv('.env.local')

True

In [9]:

def get_embedding(text, model="large"):
    if model == 'large' : 
      emb_model = "text-embedding-3-large"
    elif model == 'small' :
        emb_model = "text-embedding-3-small"
    else: 
        emb_model = "text-embedding-ada-002"
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY") )
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=emb_model).data[0].embedding


In [10]:
def update_example(user_question, list_of_obstacles, actions):
    examples_template = {
    "user_question": user_question.strip(),
    "list_of_obstacles": str(list_of_obstacles),
    "actions" : str(actions),
    "embedding_question" : get_embedding(user_question, "ada"),
}
    try : 
        df = pd.read_pickle(os.path.join("data", "examples.pkl"))
    except FileNotFoundError:
        df = pd.DataFrame(columns = examples_template.keys())
    df_new = pd.DataFrame( [examples_template ], columns = examples_template.keys())
    df = pd.concat([df, df_new], ignore_index=True)
    df.to_pickle(os.path.join("data", "examples.pkl"))

In [11]:
examples = list() 

user_question = "The robot should create an half circle with a radius of 2 meters"
list_of_obstacles = "The room has 4 walls that surround the room and a squared table with opposite vertex (x=6.5,y=5.5) and (x=8.5,y=7.5)."
actions =""" "actions" : [
    "The first action is to MOVE BACKWARD of 1.5 meters to avoid the table during the half circle creation. The robot reaches the point (x=4, y=5.5).",
    "The second action is to TURN RIGHT of 90 degrees at constant speed to create the half circle (the orientation is along x-decreasing). The robot reaches the point (x=4, y=1.5). The total path is 2*PI*R / 2 with R=2 meters that means 6.28 m that means a speed of 3.14 m/s. for 2 seconds.",
    "The third action is to ROTATE RIGHT of 90 degrees to reach the orientation along y-increasing.(x=4, y=1.5) "
    "The last action is to MOVE FORWARD of 4 meters to reach the point (x=4, y=5.5) and complete the trajectory."
    ]"""

update_example(user_question, list_of_obstacles, actions)

In [17]:
df = pd.read_pickle(os.path.join("data", "examples.pkl"))
df

##remove the second example
# df = df.drop(1)
# df
#df.to_pickle(os.path.join("data", "examples.pkl"))


,user_question,list_of_obstacles,actions,embedding_question,distance
0,"The robot must reach the point (10, 10) and c...",The room has 4 walls that surround the room,"""actions"" : [\n ""The first action is to MO...","[-0.003744511865079403, -0.0452059768140316, -...",0.560633
2,The robot should create a rhombus with angles ...,The room has 4 walls that surround the room.,"""actions"" : [\n ""The first action is ROTAT...","[0.0016683696303516626, -0.012177547439932823,...",0.500631
3,The robot should create an half circle with a ...,The room has 4 walls that surround the room an...,"""actions"" : [\n ""The first action is to MO...","[0.008686479181051254, 0.011246704496443272, -...",NaN


In [56]:
def retrive_nearest_question(user_question):
    df : pd.DataFrame = pd.read_pickle(os.path.join("data", "examples.pkl"))
    user_question_embedding = get_embedding(user_question, "ada")
    df = pd.read_pickle(os.path.join("data", "examples.pkl"))
    df["distance"] = df.embedding_question.apply(lambda x : euclidean_distances([user_question_embedding], [x])[0][0])
    return df.sort_values("distance").iloc[0].user_question
    
print(retrive_nearest_question("The robot must go out from the cage"))

The robot should exit from the house.


In [20]:
from pprint import pprint

print(""" As an expert in robotics programming, your task is to create a  sequence of commands to control the trajectory of a robot simulation, based on the following requirements:
The field is a square measuring 11.0 meters in both the x and y directions.
The robot’s starting point is at x=0 and y=0, with the head oriented along the positive x-axis (forward direction).
Each JSON object in the sequence should contain the following fields:
	    - reasoning: reason behind the movement
	    - frontal_speed: speed of forward movement, in m/s
	    - angular_speed: angular speed of anti-clockwise rotation, in degrees per second
	    - time: The duration of a single movement,in seconds
To make the robot curve its trajectory, combine both linear and angular speeds.
Do not include any comments as they may cause the application to crash.
Example : 
user : create a reversed U 
assistant : [{'reasoning' : 'rotate of 90° to go upwards with an angular speed on 45  degree/s for 2.0 second.',
      'frontal_speed' : 0.0,'angular_speed' : 45.0,'time' : 2.0 },
      'reasoning' : 'go forward at 1 m/s for 2 second to have a line of 2 m','frontal_speed' : 1.0,'angular_speed' : 0.0,'time' : 2.0},
      {'reasoning' : 'To create the curve of the U i to rotate of 90° clockwise maintaining a linear speed of 1 m/s for at least 2 second',
      'frontal_speed' : 1.0,'angular_speed' : -90.0,'time' : 2.0 },{ 'reasoning' :  'to reach the goal go forward for 1 s at 2 m/s',
      'frontal_speed' : 2.0,'angular_speed' : 0.0,'time' : 1.0}]

""")

 As an expert in robotics programming, your task is to create a  sequence of commands to control the trajectory of a robot simulation, based on the following requirements:
The field is a square measuring 11.0 meters in both the x and y directions.
The robot’s starting point is at x=0 and y=0, with the head oriented along the positive x-axis (forward direction).
Each JSON object in the sequence should contain the following fields:
	    - reasoning: reason behind the movement
	    - frontal_speed: speed of forward movement, in m/s
	    - angular_speed: angular speed of anti-clockwise rotation, in degrees per second
	    - time: The duration of a single movement,in seconds
To make the robot curve its trajectory, combine both linear and angular speeds.
Do not include any comments as they may cause the application to crash.
Example : 
user : create a reversed U 
assistant : [{'reasoning' : 'rotate of 90° to go upwards with an angular speed on 45  degree/s for 2.0 second.',
      'frontal_sp